In [1]:
import requests
from bs4 import BeautifulSoup
import time
import html2text
import pandas as pd
import random
import ware_house
from time import sleep, strftime
import re
desktop_agents = ware_house.desktop_agents
# fghg

In [2]:
def replace_other(text):
        """This function replace all not necessary symbols in the text"""
        for i in ware_house.replace_list:
            text = text.replace(i, "")

        return (text)


def random_headers():
    """This function create a random header for my parser"""
    return {'User-Agent': random.choice(desktop_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}


def handle_text(text):
    """This function handle text and make it in the good format"""
    h = html2text.HTML2Text()
    h.ignore_links = True
    h.escape_all = True
    text = h.handle(text)
    text = replace_other(text)
    return (text)


def request_action(url, timeout):
    r = requests.get(url, headers=random_headers(), timeout=timeout)
    soup = BeautifulSoup(r.text, 'html.parser')
    return (soup)

created rel -- пользователь имя

js__commentText user-used clear -- все комментарии
<b class="_12 _lgr">14 января 2016, 20:28</b> 00  ---- дата
blog-created rel fl -- время

<div class="_12 _lgr _nw fl">Латвия, Рига</div> -- местоположение 

social-repost__icon _eye _cd js__viewsCounter -- количество просмотров (у первого, потом у всех the same)

<span class="social-repost__count" rel="cnt">2</span> -- количество лайков

type_post = 






# Скрипт для парсинга 1ой странички с сайта babyblog.ru
## Нужно будет доделать:
###           1. Считывание количества лайков
###           2. Считывание количества просмотров

In [15]:
# 
# df_main = pd.read_excel('SQL\\Excel_backups\\'+'Дюфастон_20181210-215700.xlsx') 
# all = len (df_main)
# filter = len(df_main [df_main ["Href_post"].str.contains("/community/")])
df_main = df_main [df_main ["Href_post"].str.contains("/community/")]
# print (filter/all)

In [16]:
# import pandas as pd
# for index, row in df1.iterrows():
#    print (row['Author_name'], row['Author_name'])
import time
def create_df_again ():
    dictio = {        'Author_name': user_name_list,
                  'Href_user': user_href_list,
                  'Post_data': data_of_post_list,
                  'Href_post': web_site_main_list,
                  'Geography_user': geography_of_user_list,
                  'Full text': full_text_comment_list
                 }
    df1 = pd.DataFrame.from_dict(dictio)
    
    return (df1)
def save_df (df1):
    df1.to_excel('SQL\\Excel_backups\\'+'Дюфастон_20181210-comments.xlsx')
def query (url, df1):
    try:
        soup = request_action(url, 20)
    except:
        time.sleep(60)
        print ('TimeOut error')
        try:
            soup = request_action(url, 30)
        except:
            save_df(df1)
    return soup

In [20]:
user_name_list = [] #+
user_href_list = [] #+
data_of_post_list = [] #+
geography_of_user_list = [] 
type_of_comment_list = [] 
number_of_post_views_list = []
number_of_post_likes_list = []
full_text_comment_list = [] #+
web_site_main_list = []
count = 0

df1 =  create_df_again() #pd.DataFrame.from_dict(dictio)



for index, row in df_main.iterrows():
    sub_web_sites = []
    web_page =  "https://www.babyblog.ru" + row['Href_post']
    soup = query (web_page, df1)
    count += 1 
    if count % 500 == 0:
        print (index)
        df1.to_excel('SQL\\Excel_backups\\'+'Comments'+ '_v' + str(index) + '.xlsx')
        df1 =  create_df_again()
    
    if not list(soup.find_all('div', attrs={'class': "pages2-list clearfix"})):
        sub_web_sites.append ("javascript://")
    else:
        spanchik = (list(soup.find_all('div', attrs={'class': "pages2-list clearfix"})[0].children)[1]).findAll("a")
        for i in spanchik:
            sub_web_sites.append (i.attrs['href'])
        
    for web_site in sub_web_sites:
#         print (web_page)
        user_name_list.append (row['Author_name']) #Author_name
        user_href_list.append ('None_main_author') #None_main
        geography_of_user_list.append ('None_main') 
        data_of_post_list.append (row['Time_of_post']) #Time_of_posts
        
        if web_site == "javascript://":
            pass
        else:
            web_page =  "https://www.babyblog.ru" + web_site
            soup = query (web_page, df1)

        
        spans = soup.find_all('div', attrs={'class': "container x84 fl"}) 

        for i in list (spans [0].children)[9].findAll("div"):
            if 'class' in (i.attrs):
                if 'user-used' in i.attrs['class']: # достаём полный комментарий

                    full_text_comment_list.append (i.getText())
                    web_site_main_list.append (web_page)
                    geography_of_user_list.append("None")

                if '_12' in i.attrs['class'] and  '_lgr' in i.attrs['class']:

                    geography_of_user_list.pop() # маленький костылёк
                    geography_of_user_list.append(list(i.children)[0])

                if 'posted' in i.attrs['class'] and  'mb1' in i.attrs['class']: # достаём полный комментарий

                    user_href_list.append(list(i.children)[1].attrs['href']) # ссылки на всех user
                    if 'user-name' in (list(i.children)[1].attrs['class']): # достаём имена всех user отсюда

                        string = str(list(i.children)[1].get_text())
                        data_of_post_list.append (list(i.children)[3].get_text())
    #                     print ()

                        if 'Видели на сайте' in string:
                            user_name_list.append (string.split(' ', 1)[0]) # достаём имена всех user отсюда
                        else:
                            user_name_list.append (string)
        geography_of_user_list.pop()
        
        if (len(full_text_comment_list)-len(user_name_list)) != 0 or (len(geography_of_user_list)-len(user_name_list)) != 0:
            print (web_page)
            geography_of_user_list = geography_of_user_list[:len(user_name_list)]
            full_text_comment_list = full_text_comment_list[:len(user_name_list)]
            web_site_main_list = web_site_main_list[:len(user_name_list)]
        
        dictio_1 = {        'Author_name': user_name_list,
              'Href_user': user_href_list,
              'Post_data' : data_of_post_list,
              'Href_post': web_site_main_list,
              'Geography_user': geography_of_user_list,
              'Full text': full_text_comment_list
             }

        df2 = pd.DataFrame.from_dict(dictio_1)


        df1 = pd.concat([df1, df2])

        user_name_list = []
        user_href_list = []
        data_of_post_list = []
        geography_of_user_list = []
        type_of_comment_list = []
        number_of_post_views_list = []
        number_of_post_likes_list = []
        full_text_comment_list = []
        web_site_main_list = []

        
# print (len(full_text_comment_list))  # больше на 1, так как нужно добавить к тем, \
#                                    # #что ниже доп. информацию о user

# # # Сейчас есть и работает:
# print (len(user_name_list))
# print (len(geography_of_user_list))
# print (len(user_href_list))
# print (len(data_of_post_list))


https://www.babyblog.ru/community/post/conception/3420799?srcid=327157
https://www.babyblog.ru/community/lenta/conception/29874
https://www.babyblog.ru/community/lenta/conception/26140
https://www.babyblog.ru/community/lenta/conception/22408
https://www.babyblog.ru/community/lenta/conception/22407
https://www.babyblog.ru/community/lenta/conception/22406
https://www.babyblog.ru/community/lenta/conception/14938
https://www.babyblog.ru/community/lenta/conception/3
https://www.babyblog.ru/community/lenta/conception/2
https://www.babyblog.ru/community/lenta/conception/1
TimeOut error
TimeOut error
3
https://www.babyblog.ru/community/post/sterility/3200633
8


C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://ru.wikipedia.org/wiki/Тержинан%20-%20почитайте%20вот%20здесь,%20станет%20ясно,%20как%20именно%20он%20помогает%20предотвратить%20воспаление.%20По%20поводу%20рассосаться%20-%20зависит%20от%20природы%20кисты,если%20желтого%20тело-%20обязательно%20рассосется
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


https://www.babyblog.ru/community/post/conception/3397029
https://www.babyblog.ru/community/post/conception/3396943
https://www.babyblog.ru/community/post/conception/3396303
https://www.babyblog.ru/community/post/living/3217609
https://www.babyblog.ru/community/post/living/3217609/1#comm_start
https://www.babyblog.ru/community/post/conception/3395259
https://www.babyblog.ru/community/post/sterility/3199679
https://www.babyblog.ru/community/post/conception/3394925
https://www.babyblog.ru/community/post/conception/3394229
https://www.babyblog.ru/community/post/happypregn/3397569
https://www.babyblog.ru/community/post/happypregn/3397517
https://www.babyblog.ru/community/post/sterility/3198961
https://www.babyblog.ru/community/post/Baby/3142119
https://www.babyblog.ru/community/post/conception/3393065
https://www.babyblog.ru/community/post/eko-beremennost/3351589
https://www.babyblog.ru/community/post/conception/3392639
https://www.babyblog.ru/community/post/conception/3392109
https://www.

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'http://03online.com/news/dyufaston_pri_beremennosti_i_rvota/2016-5-29-168785
Посмотри%20вот%20здесь.%20Там%20можно%20чужие%20записи%20поискать%20похожие,%20а%20можно%20самой%20задать%20вопрос%20гинекологу.%20Я%20как-то%20раз%20пробовала-очень%20помогла%20Молчанова%20Е.В.%20Только%20надо%20все%20поподробней%20описывать.
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


https://www.babyblog.ru/community/post/conception/3387887
https://www.babyblog.ru/community/post/sterility/3195237
https://www.babyblog.ru/community/post/conception/3385841
https://www.babyblog.ru/community/post/conception/3385349
https://www.babyblog.ru/community/post/conception/3384889
https://www.babyblog.ru/community/post/bazal/3248533
https://www.babyblog.ru/community/post/conception/3382985
https://www.babyblog.ru/community/post/kopilkatestov/1884751
https://www.babyblog.ru/community/post/conception/3382327
https://www.babyblog.ru/community/post/sterility/3191693
https://www.babyblog.ru/community/post/conception/3382001
https://www.babyblog.ru/community/post/conception/3381975
https://www.babyblog.ru/community/post/sterility/3191577
https://www.babyblog.ru/community/post/conception/3381755
https://www.babyblog.ru/community/post/living/3212107
https://www.babyblog.ru/community/post/living/3212107/1#comm_start
https://www.babyblog.ru/community/post/conception/3381365
https://www.ba

...

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'http://www.babyblog.ru/bbstore/bazal/cycle/kaplun0va/34333эта%20Б%20была%20беспроблемной%20вообще%20и%20без%20медикаментов%20(но%20это%20не%20значит,%20что%20я%20вам%20рекомендую%20таблетки%20не%20пить,%20если%20нужно)))..,%20а%20 2%20Б%20сыном%20была%20полность%20на%20дюфастоне.. %20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'http://www.babyblog.ru/bbstore/bazal/cycle/kaplun0va/34333эта%20Б%20была%20беспроблемной%20вообще%20и%20без%20медикаментов%20 (но%20это%20не%20значит,%20что%20я%20вам%20рекомендую%20таблетки%20не%20пить,%20если%20нужно)))..,%20а%20 2%20Б%20сыном%20была%20полность%20на%20дюфастоне.. %20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\ProgramD

https://www.babyblog.ru/community/post/conception/1897395
https://www.babyblog.ru/community/post/conception/1896751
https://www.babyblog.ru/community/post/conception/1896071
https://www.babyblog.ru/community/post/conception/1896071/1#comm_start
https://www.babyblog.ru/community/post/conception/1895351
https://www.babyblog.ru/community/post/obraz_live/1754099
https://www.babyblog.ru/community/post/conception/1893261
https://www.babyblog.ru/community/post/conception/1893135
https://www.babyblog.ru/community/post/conception/1892133
https://www.babyblog.ru/community/post/conception/1891549
https://www.babyblog.ru/community/post/conception/1891549/1#comm_start
https://www.babyblog.ru/community/post/conception/1891491
https://www.babyblog.ru/community/post/conception/1891491/1#comm_start
https://www.babyblog.ru/community/post/obraz_live/1752935
https://www.babyblog.ru/community/post/conception/1890459
https://www.babyblog.ru/community/post/living/1793039
https://www.babyblog.ru/community/pos

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'http://wantbaby.ru/art/uroven-hgch-po-dnjam-ot-zachatija
Хорошая%20табличка.%20По%20ней%20ориентируюсь.%20
Хотя%20получила%20результат%20хгч%20на%2025%20дпо%20-%207307,%20вроде%20соответствует,а%20Г%20назначила%205000%20прегнила.....говорит%20-%20угроза.....((((%20ей%20виднее%20наверное...укололи...
 ' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


https://www.babyblog.ru/community/post/conception/1878355
https://www.babyblog.ru/community/post/obraz_live/1748601
https://www.babyblog.ru/community/post/conception/1875819
https://www.babyblog.ru/community/post/conception/1875819/1#comm_start
https://www.babyblog.ru/community/post/conception/1875337
https://www.babyblog.ru/community/post/living/1782809
https://www.babyblog.ru/community/post/happypregn/1831507
https://www.babyblog.ru/community/post/living/1780281
https://www.babyblog.ru/community/post/conception/1866899
4
https://www.babyblog.ru/community/post/conception/1859519
https://www.babyblog.ru/community/post/sterility/1754007
https://www.babyblog.ru/community/post/living/1772735
https://www.babyblog.ru/community/post/living/1772735/1#comm_start
https://www.babyblog.ru/community/post/sterility/1752881
https://www.babyblog.ru/community/post/conception/1851071
https://www.babyblog.ru/community/post/living/1769083
https://www.babyblog.ru/community/post/living/1769083/1#comm_start

IndexError: list index out of range

In [21]:
df1.to_excel('SQL\\Excel_backups\\'+'Comments'+ '_other.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'http://www.piluli.kharkov.ua/drugs/drug/acidum-folicum/%20 %20 выдержка%20из%20инструкции,полная%20инструкция%20по%20ссылкеДополнительно:При%20анемии%20пернициозной(В12%20анемия)%20фолиевую%20кислоту%20назначают%20только%20с%20цианокобаламином(В12),%20так%20как%20последний%20способствует%20предупреждению%20неврологических%20нарушений.%20При%20спру%20фолиевую%20кислоту%20назначают%20с%20аскорбиновой%20кислотой%20и%20цианокобаламином.%20Фолиевая%20кислота%20может%20включаться%20в%20поливитаминные%20препараты.%20При%20железодефицитных%20анемиях%20в%20дополнение%20к%20приему%20фолиевой%20кислоты%20необходим%20железосодержащий%20препарат.%20Неэффективен%20прием%20фолиевой%20кислоты%20для%20лечения%20апластической%20анемии.%20Не%20предупреждает%20неврологические%20осложнения%20(например,%20фуникулярный%20миелоз%20и%20другие).%20В%20больших%20дозах%20не%20рекомендуется%20принимать%20препарат,%20